# GCS model in Python
## Solving Gouy-Chapman-Stern (GCS) numerically from scratch

In today's exercise, we will seek to solve the Gouy-Chapman-Stern Boundary Value Problem (BVP) numerically:

$$
\text{GCS (1D):} 
\begin{cases} 
\frac{\text{d}^{2}\phi}{\text{d}x^{2}} = \frac{F c_{b}}{\epsilon_{s}}\sinh(F\frac{\phi}{RT}), & X \in ]0, L_{\text{max}}[ \\
\phi(x = L_{\text{min}}) = \phi_{\text{HP}}, & X = L_{\text{min}} \tag{1} \\
\lim_{x \to L_{\text{max}}} \phi(x = L_{\text{max}}) = 0, & X \to L_{\text{max}}
\end{cases}
$$

where F is the Faraday constant, $\approx 9.649\cdot 10^{4} \text{ C}\cdot\text{mol}^{-1}$, $\epsilon_{\text{S}}$ is the relative permittivity (old: dielectric constant) of the solvent with respect to electric permittivity of a vacuum, $\epsilon_{\text{S}}\approx 78.4$ (for pure water), $c_{b}$ is the bulk concentration (Molar, M or millimolar, mM), $\phi$ is the electric potential (Volt, V), $x$ is the distance from the Helmholtz Plane (nm).
<p><p>
We will adopt the Newton-Raphson method and benchmark (compare) using the analytical solution to the problem with boundary conditions for $L_{\text{min}} = 0$ and $L_{\text{max}} \to \infty$:

$$
\begin{align}
\phi = 2\, \ln\left(\frac{\sqrt{\exp\left(\phi_{\text{HP}}\right)} - \tanh\left(-x \sqrt{\frac{2 F^{2}c_{b} }{RT\epsilon_{s}}}\right) }{1 - \sqrt{\exp\left(\phi_{\text{HP}}\right)}\tanh\left(-x \sqrt{\frac{2 F^{2}c_{b} }{RT\epsilon_{s}}}\right) }\right) \tag{2}
\end{align}
$$

where $R$ is the gas constant, $\approx 8.3145 \text{ J}\cdot \text{mol}^{-1}\cdot\text{K}^{-1}$, and $T$ is the temperature (Kelvin, K).

### Setup parameters

The first step, and probably the easiest, is to define the parameters of the model. This can be done in a simple script. But let's define it using classes. This way, we can also use getters (get the values) and setters (restrictions imposed using physical intuition). Example: We get the temperature. And we know that the temperature, in Kelvin, can only take positive real numbers. If at any point we impose a negative value temeprature, we immediatly raise an error.

In [3]:
class Params:
    
    # Valid constructor arguments
    valid_keys = ['T', 'epsilon_s', 'concentration_mM', 'phi_M', 'phi_pzc', 'N', 'L_min', 'L_max']
    
    r"""
    Parameters for the Gouy-Chapman-Stern (GCS) model, including fundamental physical constants.

    Fundamental constants (fixed for all simulations):
    ---------------------------------------------------
    - kB : float
        Boltzmann constant \((k_B = 1.381 \times 10^{-23} \ \text{J/K})\).
    - q : float
        Elementary charge \((q = 1.602 \times 10^{-19} \ \text{C})\).
    - NA : float
        Avogadro's number \((N_A = 6.02214076 \times 10^{23} \ \text{mol}^{-1})\).
    - R : float
        Gas constant \((R = 8.3145 \ \text{J/mol·K})\).
    - F : float
        Faraday constant \((F = 9.649 \times 10^4 \ \text{C/mol})\).
    - epsilon0 : float
        Vacuum permittivity \((\epsilon_0 = 8.854 \times 10^{-12} \ \text{F/m})\).

    Simulation parameters (user-defined):
    -------------------------------------
    - T : float
        Temperature in Kelvin.
    - epsilon_s : float
        Relative permittivity of the medium.
    - concentration_mM : float
        Bulk electrolyte concentration in millimolar (mM).
    - phi_M : float
        Electrode potential (Volt).
    - phi_pzc : float
        Point of zero charge with respect to the electrode potential (Volt).
    - N : int
        Number of grid points in the spatial domain.
    - L_min : float
        Minimum spatial domain length in meters.
    - L_max : float
        Maximum spatial domain length in meters.

    Derived attributes:
    --------------------
    - c_b_mol : float
        Bulk concentration converted from millimolar to molar (mol/m³).
    - n_b : float
        Number density of particles (1/cm³).
    """
    
    def __init__(self, T=298.15, epsilon_s=78.4, concentration_mM=100.0, phi_M = 0.0,
                        phi_pzc=0.0, N=2**6, L_min=0.0, L_max = 5e-9):
        """
        Initialize simulation parameters for the GCS model.

        Sets physical constants, simulation domain bounds, and computes
        derived properties such as molar concentration and particle density.
        """
        
        # Fundamental constants
        self.kB = 1.381e-23      # Boltzmann constant [J/K]
        self.q = 1.602e-19       # Elementary charge [C]
        self.NA = 6.02214076e23  # Avogadro's number [1/mol]
        self.R = 8.3145          # Gas constant [J/mol·K]
        self.F = 9.649e4         # Faraday constant [C/mol]
        self.epsilon0 = 8.854e-12  # Vacuum permittivity [F/m]

        # Parameters
        self.T = T
        self.epsilon_s = epsilon_s
        self.concentration_mM = concentration_mM
        self.phi_M = phi_M
        self.phi_pzc = phi_pzc
        self.N = N
        
        # Retrieved parameters
        self.set_domain_bounds(L_min, L_max)
        self.c_b_mol = self.convert_mM_to_M(concentration_mM)
        self.n_b = self.calculate_particle_density(self.c_b_mol)
    
    @property
    def T(self):
        """Return the temperature in Kelvin."""
        return self._T

    @T.setter
    def T(self, value):
        """
        Set the temperature (Kelvin).

        Parameters
        ----------
        value : float
            The temperature in Kelvin. Must be greater than zero.

        Raises
        ------
        ValueError
            If the temperature is less than or equal to zero.
        """
        if value <= 0:
            raise ValueError("Temperature must be positive.")
        self._T = value

    @property
    def epsilon_s(self):
        """Return the relative permittivity of the medium."""
        return self._epsilon_s

    @epsilon_s.setter
    def epsilon_s(self, value):
        """
        Set the relative permittivity.

        Parameters
        ----------
        value : float
            The relative permittivity of the medium. Must be greater than zero.

        Raises
        ------
        ValueError
            If the relative permittivity is less than one.
        """
        if value < 1:
            raise ValueError("Permittivity must be greater than 1.")
        self._epsilon_s = value

    @property
    def concentration_mM(self):
        """Return the bulk concentration in millimolar."""
        return self._concentration_mM

    @concentration_mM.setter
    def concentration_mM(self, value):
        """
        Set the bulk concentration in millimolar.

        Parameters
        ----------
        value : float
            The concentration in millimolar (mM). Must be greater than zero.

        Raises
        ------
        ValueError
            If the concentration is less than or equal to zero.
        """
        if value <= 0:
            raise ValueError("Concentration must be positive.")
        self._concentration_mM = value

    @property
    def N(self):
        """Return the number of grid points in the spatial domain."""
        return self._N

    @N.setter
    def N(self, value):
        """
        Set the number of grid points.

        Parameters
        ----------
        value : int
            The number of grid points. Must be a positive integer.

        Raises
        ------
        ValueError
            If the number of grid points is not a positive integer.
        """
        if not isinstance(value, int) or value <= 0:
            raise ValueError("Number of grid points must be a positive integer.")
        self._N = value
    
        
    @property
    def L_min(self):
        """Return the minimum length of the domain in meters."""
        return self._L_min

    @property
    def L_max(self):
        """Return the maximum length of the domain in meters."""
        return self._L_max
    
    def set_domain_bounds(self, L_min, L_max):
        """
        Set the spatial domain boundaries for the simulation.

        This method assigns the minimum and maximum lengths of the spatial domain
        and checks that the values are physically valid. It ensures that `L_min` is
        strictly less than `L_max` to avoid degenerate or reversed domains.

        Parameters
        ----------
        L_min : float
            The minimum boundary of the spatial domain, in meters.
        L_max : float
            The maximum boundary of the spatial domain, in meters.

        Raises
        ------
        ValueError
            If `L_min` is greater than or equal to `L_max`.
        """
        if L_min >= L_max:
            raise ValueError("L_min must be less than L_max.")
        self._L_min = L_min
        self._L_max = L_max

    def convert_mM_to_M(self, concentration_mM):
        """Converts concentration from mM to M."""
        return concentration_mM * 1e-3

    def calculate_particle_density(self, concentration_molar):
        """Calculates number density of particles (1/cm³)."""
        return concentration_molar * self.NA

### Parameter Class check
If we do not check if what we are doing works, then we cannot really blame anyone but ourselves, if the code does not work down the line. Let us check if this satistifies our expectations. For this reason, I want you to answer the following questions using the code snippet below:

- What would you do in order to fix the code snippet? 
- How would you change the number of grid points to 256? 
- What would do, if you **only** wanted default values?

In [5]:
# Check if the parameters are allowed

#----- YOUR INPUT -----#
T = -298

L_minnn = 0

L_max = -5e-9

# ?

#----- YOUR INPUT -----#

In [4]:
# Get all valid keys based on the params class
valid_keys = Params.valid_keys  # Defined inside the class
inputs = {key: globals()[key] for key in valid_keys if key in globals()}

# Employ try and except to show succesfull values
# And tell user if any errors are arised.
try:
    #overwrite 
    params = Params(**inputs)
    
    #compare with default values
    default = Params()
    
    print("Params object created successfully.")
    print("Overwritten parameters:")
    for key in valid_keys:
        val = getattr(params, key)
        if key in inputs:
            print(f"  {key} = {val}  (user-defined)")
        else:
            print(f"  {key} = {val}  (default)")
except ValueError as e:
    print(f"Caught an error: {e}")
except TypeError as e:
    print("TypeError: you misspelled a parameter name")
    print("", e)

Params object created successfully.
Overwritten parameters:
  T = 298.15  (default)
  epsilon_s = 78.4  (default)
  concentration_mM = 100.0  (default)
  phi_M = 0.0  (default)
  phi_pzc = 0.0  (default)
  N = 64  (default)
  L_min = 0.0  (default)
  L_max = 5e-09  (default)


# From real world to numerical
The Newton<span>&ndash;</span>Raphson method is a method used to solve nonlinear differential equations. But in order to employ it, the differential equation in question, eq. (1), ought to be discretized.

## Discretization
Everything in the physical world is either continous and smooth, discrete or somewhere inbetween. Unfortunately, computers only understadnd the discretized world. Therefore, if we wish to numerically/ computationally solve GCS (or any differential equation for that matter), we must employ a method, which rewrites any $\frac{d^{2} \phi}{\text{d}x^{2} }$ terms in a way that the computer will understand. This is called discretization. For today's exercises, we employ a Second Order Central Difference discretization. Let $N$ be the amount of grid points and $i\in {0,1,2, \cdots, N -1}$ and  assume that our $x$ is uniformly distributed: $h = x_{i+1} - x_{i}$ for $0\geq i \geq N - 2$, then the difference is found by the following expression: $\frac{d^{2} \phi}{\text{d}x^{2} } \approx \frac{\phi\left(x_{i}+h\right) - 2\phi\left(x_{i}\right) + \phi\left(x_{i} - h\right) }{h^{2} }$. In this way, eq. (1) now becomes:
$$
    \frac{\phi\left(x_{i}+h\right) - 2\phi\left(x_{i}\right) + \phi\left(x_{i} - h\right) }{h^{2} } = \frac{F c_{b}}{\epsilon_{s}}\sinh(\phi(x_{i})) \tag{3}
$$
though, because we want to list the $\phi$ values as list anyway, we may instead impose the following way of counting: $\cdots, \phi^{(i-1)} = \phi\left(x_{i} - h\right),\phi^{(i)} = \phi\left(x_{i}\right), \phi^{(i+1)} = \phi\left(x_{i} + h\right), \cdots$. This way, we can define an important property, namely the residual:
$$
    F_{i} = F\left(\phi^{(i)}\right) = \frac{\phi^{(i-1)} - 2\phi^{(i)} + \phi^{(i+1)}  }{h^{2} } - \frac{F c_{b}}{\epsilon_{s}} \sinh\left(\phi^{(i)}\right) \tag{4}
$$
Which is $just$ the Left-Hand-Side (internal term) substracted by the Right-Hand-Side (external/ source term). If $F_{i} \approx 0$ for all $i$, we have solved the problem. We need to calculate a value that encapsulates the overall assessment of the error being close to $0$. A good choice is the euclidean norm:
$$
    ||\mathbf{F}||_{2} = \sqrt{F_{1}^{2} + F_{2}^{2} + \cdots + F_{N}^{2}} \tag{5}
$$
If this norm is smaller than a preset tolerance value (in this exercise we choose $10^{-10}$): $||\mathbf{F}||_{2} < \varepsilon_{\text{error}}$, we say that our simulation has reached convergence and plot the $\phi$ or use it for post-processing. Now, that we have a goal, we just need a means of procedure. This is where Newton-Raphson comes in. 


## Newton<span>&ndash;</span>Raphson method
In order to reach convergence, we need a procedure which changes the $\phi$-values for better or worse. The Newton-Raphson procedure proposes the following method:
$$
    \mathbf{J}\Delta\phi = \mathbf{F} \tag{6}
$$
which you may recognize as a linear system. $J$ is the Jacobian defined by:
$$
    J_{ij} = \frac{\text{d} F_{i}}{\text{d} \phi^{(j)}} \tag{7}
$$
and just like before, differential quotients cannot be true in numerical/ computational studies. Again, we impose a second order central difference method, this time applied to a first order derivative:
$$
    J_{ij} \approx \frac{F_{i}\left(\phi_{1}, \phi_{2}, \cdots, \phi_{j} + \epsilon, \cdots, \phi_{N}\right) - F_{i}\left(\phi_{1}, \phi_{2}, \cdots, \phi_{j} - \epsilon, \cdots, \phi_{N}\right)}{2\epsilon} \tag{8}
$$
where $\epsilon$ serves the purpose as $h$ before. Finally, we can solve subtitute into eq. (6) and solve for $\Delta \phi$. If we keep doing this, updating $\mathbf{F}$ at every iteration and checking for convergence, we should have final method. We just need to update the solution using the prior iteration step $\phi^{(k)}$:
$$
\mathbf{\phi}^{(k+1)} = \mathbf{\phi}^{(k)} + \Delta \mathbf{\phi}^{(k)} \tag{9}
$$
One last question arises. How do solve the linear system provided by eq. (6)? This is beyond the scope of this course. We employ [LU decomposition](https://en.wikipedia.org/wiki/LU_decomposition), which the curious student is welcome to look up.



In [1]:
class Residual:
    """
    Class to compute the residual F(phi) for the GCS model.

    Parameters
    ----------
    params : Params
        Simulation parameters containing constants and grid information.

    Attributes
    ----------
    params : Params
        Stored parameters object for easy access.
    h : float
        Grid spacing.
    prefactor : float
        Precomputed prefactor (F * c_b) / (epsilon_r) for residual.
    """
    def __init__(self, params):
        self.params = params
        self.h = (params.L_max - params.L_min) / (params.N - 1)

        epsilon_0 = 8.854187817e-12  # Vacuum permittivity [F/m]
        self.prefactor = (9.649e4) * params.c_b_mol / (params.epsilon_s * epsilon_0)  # Use F from your constants

    def compute_residual(self, phi):
        r"""
        Compute the residual vector F(phi).

        The residual is defined by discretizing the second derivative using a second-order
        central difference scheme:
        
        $$
        F_i = \frac{\phi^{(i-1)} - 2\phi^{(i)} + \phi^{(i+1)}}{h^2} - \frac{F c_b}{\epsilon_s \epsilon_0} \sinh(\phi^{(i)})
        $$
        
        where
        $$
        \sinh(x) = \frac{e^{x} - e^{-x}}{2}
        $$

        Parameters
        ----------
        phi : list of floats
            List of phi values at the grid points.

        Returns
        -------
        F_residual : list of floats
            Residual vector.
        """
        N = self.params.N
        F_residual = [0.0 for _ in range(N)]

        # Internal points
        for i in range(1, N-1):
            F_residual[i] = (phi[i-1] - 2 * phi[i] + phi[i+1]) / (self.h**2) - self.prefactor * self.sinh(phi[i])

        # Boundary conditions
        F_residual[0] = phi[0]  # phi(0) = 0
        F_residual[-1] = phi[-1] - (self.params.phi_pzc - self.params.phi_pzc)  # phi(L) = phi_M - phi_pzc

        return F_residual

    def sinh(self, x):
        r"""
        Compute sinh(x) manually without numpy.

        The hyperbolic sine function is defined as:
        
        $$
        \sinh(x) = \frac{e^{x} - e^{-x}}{2}
        $$

        Parameters
        ----------
        x : float

        Returns
        -------
        sinh(x) : float
        """
        from math import exp
        return (exp(x) - exp(-x)) / 2


In [2]:
class Jacobian:
    """
    Class to compute the Jacobian matrix J(phi) for the GCS model.

    Parameters
    ----------
    residual_calculator : Residual
        A Residual object used to compute F(phi).

    Attributes
    ----------
    residual_calculator : Residual
        Stored Residual object.
    params : Params
        Parameters associated with the Residual.
    epsilon : float
        Perturbation size for numerical differentiation.
    """
    def __init__(self, residual_calculator, epsilon=1e-8):
        self.residual_calculator = residual_calculator
        self.params = residual_calculator.params
        self.epsilon = epsilon  # finite difference step for Jacobian, NOT h

    def compute_jacobian(self, phi):
        r"""
        Compute the Jacobian matrix J evaluated at phi.

        The Jacobian elements are computed using a second-order central difference formula:
        
        $$
        J_{ij} = \frac{F_i(\phi_1, \ldots, \phi_j + \epsilon, \ldots, \phi_N) - F_i(\phi_1, \ldots, \phi_j - \epsilon, \ldots, \phi_N)}{2\epsilon}
        $$

        Parameters
        ----------
        phi : list of floats
            List of phi values at the grid points.

        Returns
        -------
        J_matrix : list of lists of floats
            Jacobian matrix of shape (N, N).
        """
        N = self.params.N
        # Initialize empty Jacobian matrix
        J_matrix = [[0.0 for _ in range(N)] for _ in range(N)]
        
        for j in range(N):
            # Create perturbed copies of phi
            phi_forward = phi.copy()
            phi_backward = phi.copy()

            phi_forward[j] += self.epsilon
            phi_backward[j] -= self.epsilon

            # Compute residuals at perturbed phi
            F_forward = self.residual_calculator.compute_residual(phi_forward)
            F_backward = self.residual_calculator.compute_residual(phi_backward)

            # Central difference for the j-th column
            for i in range(N):
                J_matrix[i][j] = (F_forward[i] - F_backward[i]) / (2 * self.epsilon)

        return J_matrix


In [3]:
# Linear system solver (you do not need to understand this)
class LUSolver:
    '''
    A class for solving linear systems of the form J Δϕ = -F
    using LU decomposition with partial pivoting.

    Parameters
    ----------
    A : list of list of floats
        The square coefficient matrix J of the system.

    Methods
    -------
    lu_decomposition(A)
        Computes L and U matrices such that A = L * U, applying partial pivoting.

    forward_substitution(b)
        Solves L y = b for y.

    backward_substitution(y)
        Solves U x = y for x.

    solve(F)
        Solves the full system J Δϕ = -F and returns Δϕ.

    Notes
    -----
    - Partial pivoting is applied for numerical stability.
    - LU decomposition is performed at initialization.
    - Solving is split into two simple triangular solves: forward and backward.
    '''
    
    def __init__(self, A):
        self.L, self.U = self.lu_decomposition(A)

    def lu_decomposition(self, A):
        n = len(A)
        L = [[0.0 for _ in range(n)] for _ in range(n)]
        for i in range(n):
            L[i][i] = 1.0

        U = [row[:] for row in A]

        for i in range(n):
            # Pivoting step for stability
            max_row = max(range(i, n), key=lambda r: abs(U[r][i]))
            if i != max_row:
                U[i], U[max_row] = U[max_row], U[i]
                L[i][:i], L[max_row][:i] = L[max_row][:i], L[i][:i]

            for j in range(i+1, n):
                factor = U[j][i] / U[i][i]
                L[j][i] = factor
                for k in range(i, n):
                    U[j][k] -= factor * U[i][k]

        return L, U


    def forward_substitution(self, b):
        n = len(self.L)
        y = [0.0 for _ in range(n)]

        for i in range(n):
            sum_terms = sum(self.L[i][j] * y[j] for j in range(i))
            y[i] = b[i] - sum_terms

        return y

    def backward_substitution(self, y):
        n = len(self.U)
        x = [0.0 for _ in range(n)]

        for i in reversed(range(n)):
            sum_terms = sum(self.U[i][j] * x[j] for j in range(i+1, n))
            x[i] = (y[i] - sum_terms) / self.U[i][i]

        return x

    def solve(self, F):
        minus_F = [-f for f in F]
        y = self.forward_substitution(minus_F)
        return self.backward_substitution(y)


# Dimensionless form

Instead, by employing 

$$
\text{GCS (1D):} \quad
\begin{cases}
    \frac{\text{d}^{2}U}{\text{d}X^{2}} = \sinh(U), & \quad X \in ]0, \infty[ \\
    U(0) = U_{\text{HP}}, & \quad X = 0 \tag{3} \\
    \lim_{x \to \infty} U(X) = 0, & \quad X \to \infty
\end{cases}
$$

Where the following transformations have been imposed:

- $ \phi = \frac{RT}{F} U $
- $ x = \lambda_{\text{D}}\, X $
- $ \lambda_{D} = \sqrt{\frac{RT\epsilon_{S}}{2F^{2}c_{b} }} $



Now, that we have these transformations, we simply need to get the transform the parameters and functions to dimensionless form **before** running our solver. After retrieving the solution, we will transform the dimensionless form back to non-dimensionless form and plot it. But how?
- $U = ?$
- $X = ?$

In [14]:
class Transformations:
    r"""
    A collection of transformations for electrostatics and electrolyte theory,
    based on the fundamental constants and parameters stored inside a Params object.

    Methods
    -------
    compute_thermal_voltage()
        Compute the thermal voltage \( V_T = \frac{k_B T}{q} \).

    compute_debye_length()
        Compute the Debye length \(\lambda_D\) for a symmetric monovalent electrolyte.

    nondimensionalize_potential(phi, phi_ref=0.0)
        Convert dimensional potential \(\phi\) (V) to dimensionless potential \(U\).

    dimensionalize_potential(U, phi_ref=0.0)
        Convert dimensionless potential \(U\) back to dimensional potential \(\phi\) (V).

    nondimensionalize_length(x)
        Convert dimensional length \(x\) (m) to dimensionless length \(X\).

    dimensionalize_length(X)
        Convert dimensionless length \(X\) back to physical length \(x\) (m).

    to_nanometers(x_m)
        Convert meters to nanometers.

    to_meters(x_nm)
        Convert nanometers to meters.

    dimensionless_concentration()
        Return 1 for nonzero concentration (used for scaling).
    """

    def __init__(self, params):
        """
        Initialize with a Params object.

        Parameters
        ----------
        params : Params
            Simulation parameters and constants.
        """
        self.params = params

    def compute_thermal_voltage(self):
        """Compute the thermal voltage V_T = kB * T / q."""
        return (self.params.kB * self.params.T) / self.params.q

    def compute_debye_length(self):
        """Compute the Debye length lambda_D."""
        epsilon_s = self.params.epsilon_s * self.params.epsilon0
        ionic_strength = self.params.c_b_mol
        return (epsilon_s * self.params.kB * self.params.T / (self.params.q**2 * ionic_strength))**0.5

    def nondimensionalize_potential(self, phi, phi_ref=0.0):
        """Convert dimensional potential (V) to dimensionless potential."""
        V_T = self.compute_thermal_voltage()
        return (phi - phi_ref) / V_T

    def dimensionalize_potential(self, U, phi_ref=0.0):
        """Convert dimensionless potential back to dimensional (V)."""
        V_T = self.compute_thermal_voltage()
        return U * V_T + phi_ref

    def nondimensionalize_length(self, x):
        """Convert dimensional length (m) to dimensionless length."""
        lambda_D = self.compute_debye_length()
        return x / lambda_D

    def dimensionalize_length(self, X):
        """Convert dimensionless length back to physical length (m)."""
        lambda_D = self.compute_debye_length()
        return X * lambda_D

    def to_nanometers(self, x_m):
        """Convert meters to nanometers."""
        return x_m * 1e9

    def to_meters(self, x_nm):
        """Convert nanometers to meters."""
        return x_nm / 1e9

    def dimensionless_concentration(self):
        """Return 1 for nonzero concentration, 0 otherwise."""
        c = self.params.c_b_mol
        return c / c if c != 0 else 0
